In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qrcode
from PIL import Image
import hashlib
import random

SECRET = 'rispro'
# SALT = ''.join(format(ord(x), 'x') for x in SECRET)

def generate_watermark(data,secret,quant=16,size=(100,100),rprop=100):
    dist  = [int((i+0.5)*256/quant) for i in range(quant)]    
    s = data+secret    
    seed = abs(int(hashlib.sha1(s.encode("utf-8")).hexdigest(), 16))%2**32
    random.seed(seed) 
    p=random.choices([i for i in range(rprop)], k = quant)
    watermark = random.choices(dist,weights=p,k=size[0]**2)
    watermark = np.array(watermark).reshape(size)
    return watermark.astype('uint8')

def make_secureQR(data:str,secret:str=SECRET,                  
                  pct_wm_size:int=0.3, 
                  bx_size:int=10,
                  border:int=4,
                  qr_ver:int=None,
                  **kwargs)-> np.ndarray:

    img = qrcode.make(data,version=qr_ver, box_size=bx_size, border=border, error_correction=qrcode.constants.ERROR_CORRECT_H)
    img = np.asanyarray(img).astype('uint8')*255
    
    wm_size = (img.shape[0]//bx_size - 2*border) * pct_wm_size
    wm_size = int(wm_size)
    if not wm_size%2==0: wm_size-=1 

    wm_size *= bx_size
    size = (wm_size,wm_size)
    watermark = generate_watermark(data,secret,size=size,**kwargs)
    wm = np.pad(watermark, bx_size//2, constant_values=255)

    # add border watermark 8
    length = wm.shape[0] 
    loc = img.shape[0]//2 - length//2
    img[loc:loc+length,loc:loc+length]= wm
    img[0] = 0
    img[369] = 0
    for i in range(370):
        img[i][0] = 0
        img[i][369] = 0
    return img, watermark

import cv2
import numpy as np
from scipy.special import rel_entr
from scipy.spatial import distance

def read_sqr(im):
    qrDecoder = cv2.QRCodeDetector()
    
    myqr = qrDecoder.detectAndDecode(im)
    if myqr[0]=="": 
        im_filter = np.copy(im) 
        myqr = qrDecoder.detectAndDecode(cv2.medianBlur(im_filter,ksize=3))
    
    top = myqr[1][0][0].astype(int)
    bottom = myqr[1][0][2].astype(int)+1
    qim = im[top[0]:bottom[0],top[1]:bottom[1]].astype(float)

    # # normalize
    qim -= qim.min()
    qim *= 255/qim.max()

    return myqr[0],qim

def JS_div(p,q):
    m = (p+q)/2
    div = 0.5*rel_entr(p,m) + 0.5*rel_entr(q,m)
    return div.sum()

def compute_KL(qim,scan,quant=16,pct=0.2,metric='JS'):
    pq = []
    for q in [qim,scan]:
        l = q.shape[0]
        half = l//2
        r = int(0.5*pct*l)             
        wtm = q[half-r:half+r,half-r:half+r]
        wtm = cv2.Sobel(wtm,cv2.CV_64F,1,1,ksize=3)
        dist, *note = np.histogram(wtm.ravel(),bins=quant)
        p = dist/dist.sum()
        pq.append(p)        
    return rel_entr(*pq).sum() if metric=="KL" else distance.jensenshannon(*pq), pq  

def compare2template(im, qr_ver = 3, quant=16, pct=0.2, metric='JS'):
    data ,qim_scan = read_sqr(im) 
    img,_ = make_secureQR(data, qr_ver = qr_ver, quant=quant)
    _,qim = read_sqr(img) 
    return (data,)+compute_KL(qim,qim_scan,quant,pct,metric)

In [4]:
# generate batch of images
import pathlib
import numpy as np
from pathlib import Path

def generate_batch(path):
    path =Path(path)
    rows = []
    for i in range(100): 
        print(i,end=' ')
        for j in range(2):
            col = []
            for k in range(5):
                img,wtm = make_secureQR(f'data_{i+1:03}',qr_ver = 3, quant=16)
                col.append(img)
            rows.append(np.hstack(col))
        if (i+1)%4 ==0:
            rows.append(np.ones((10,rows[0].shape[1]),dtype='uint8'))
            grid = np.vstack(rows)

            Image.fromarray(grid).save(path/f"{i-2:03}_{i+1}.pdf")
            rows=[]

In [25]:
img,wtm = make_secureQR(f'data_{1:03}',qr_ver = 3, quant=8)
# resized_qr = cv2.resize(img, (500, 500), interpolation=cv2.INTER_NEAREST)

In [27]:
img.shape

(370, 370)

In [19]:
print(img)
print(img.shape)
img[0] = 0
img[369] = 0
for i in range(370):
  img[i][0] = 0
  img[i][369] = 0

[[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
(370, 370)


In [20]:
cv2.imshow('img', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# generate batch of images
import pathlib
import numpy as np
from pathlib import Path
import cv2

def generate_batch2(path):
    path =Path(path)
    rows = []
    count = 1
    while(count <= 100):
        for j in range(2):
            col = []
            for k in range(5):
                white_space = np.ones(shape=(370,370),dtype='uint8')
                white_space.fill(255)
                col.append(white_space)
            rows.append(np.hstack(col))
        for j in range(4):
            col = []
            for k in range(5):
                if k == 0 or k == 4:
                    white_space = np.ones(shape=(370,370),dtype='uint8')
                    white_space.fill(255)
                    col.append(white_space)
                else:
                    print(f'data_{count}')
                    img,wtm = make_secureQR(f'data_{count:03}',qr_ver = 3, quant=16)
                    col.append(img)
                    count += 1
            rows.append(np.hstack(col))
        for j in range(2):
            col = []
            for k in range(5):
                white_space = np.ones(shape=(370,370),dtype='uint8')
                white_space.fill(255)
                col.append(white_space)
            rows.append(np.hstack(col))
        # rows.append(np.ones((10,rows[0].shape[1]),dtype='uint8'))
        grid = np.vstack(rows)
        Image.fromarray(grid).save(path/f"data_{count-12:03}_{count-1:03}.png")
        rows=[]
        return grid
        # Image.fromarray(grid).save("test.png")

In [6]:
# Generate QR batch A3
import pathlib
import numpy as np
from pathlib import Path
import cv2

def generate_batchA3(path):
    path =Path(path)
    rows = []
    count = 81    
    count_awal = count
    col = []
    white_space = np.ones(shape=(74,2198),dtype='uint8')
    white_space.fill(255)
    col.append(white_space)
    rows.append(np.hstack(col))
    for j in range(8):
        col = []
        white_space = np.ones(shape=(370,174),dtype='uint8')
        white_space.fill(255)
        col.append(white_space)
        for k in range(5):            
            print(f'data_{count}')
            img,wtm = make_secureQR(f'data_{count:03}',qr_ver = 3, quant=8)             
            col.append(img)
            count += 1
        white_space = np.ones(shape=(370,174),dtype='uint8')
        white_space.fill(255)
        col.append(white_space)
        rows.append(np.hstack(col))
        
    col = []
    white_space = np.ones(shape=(74,2198),dtype='uint8')
    white_space.fill(255)
    col.append(white_space)
    rows.append(np.hstack(col))
    
    grid = np.vstack(rows)
    Image.fromarray(grid).save(path/f"data_{count_awal:03}_{count-1:03}_original.png")
    rows=[]
    return grid

In [7]:
gridA3 = generate_batchA3('A3')

data_81
data_82
data_83
data_84
data_85
data_86
data_87
data_88
data_89
data_90
data_91
data_92
data_93
data_94
data_95
data_96
data_97
data_98
data_99
data_100
data_101
data_102
data_103
data_104
data_105
data_106
data_107
data_108
data_109
data_110
data_111
data_112
data_113
data_114
data_115
data_116
data_117
data_118
data_119
data_120


In [85]:
grid2 = generate_batch2('watermarks')

data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9
data_10
data_11
data_12


In [86]:
grid2.shape

(2960, 1850)

In [87]:
grid.shape

(2960, 1850)

In [7]:
img = cv2.imread('photo1/1-12.jpg',0)
img = cv2.resize(img, (0, 0), fx=0.5, fy=0.5)
cv2.imshow('cropped', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [11]:
# importing the module
import cv2
  
# function to display the coordinates of
# of the points clicked on the image
def click_event(event, x, y, flags, params):
 
    # checking for left mouse clicks
    if event == cv2.EVENT_LBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(img, str(x) + ',' +
                    str(y), (x,y), font,
                    1, (255, 0, 0), 2)
        cv2.imshow('image', img)
 
    # checking for right mouse clicks    
    if event==cv2.EVENT_RBUTTONDOWN:
 
        # displaying the coordinates
        # on the Shell
        print(x, ' ', y)
 
        # displaying the coordinates
        # on the image window
        font = cv2.FONT_HERSHEY_SIMPLEX
        b = img[y, x, 0]
        g = img[y, x, 1]
        r = img[y, x, 2]
        cv2.putText(img, str(b) + ',' +
                    str(g) + ',' + str(r),
                    (x,y), font, 1,
                    (255, 255, 0), 2)
        cv2.imshow('image', img)
 
# driver function
if __name__=="__main__":
 
    # reading the image
    img = cv2.imread('photo1/1-12.jpg', 1)
    img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)
 
    # displaying the image
    cv2.imshow('image', img)
 
    # setting mouse handler for the image
    # and calling the click_event() function
    cv2.setMouseCallback('image', click_event)
 
    # wait for a key to be pressed to exit
    cv2.waitKey(0)
 
    # close the window
    cv2.destroyAllWindows()

313   615
468   614
312   773
467   770


In [74]:
img = cv2.imread('photo1/1-12.jpg', 0)
# img = cv2.resize(img, (0, 0), fx=0.3, fy=0.3)

In [55]:
img.shape

(4624, 2608)

In [52]:
img.shape

(1387, 782)

In [57]:
cropped = img[1534:2050, 527:1044]
cropped2 = img[925:1080, 468:623]

In [23]:
cropped = []
517

In [58]:
cv2.imshow('cropped', cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

KeyboardInterrupt: 

In [75]:
count = 0
cropped = []
for i in range(4):
  for j in range(3):
    # 1534:2050, 527:1044
    cropped.append(cv2.resize(img[1533+i*517:2050+i*517, 527+j*517:1044+j*517], (370,370)))
    Image.fromarray(cropped[count]).save(f"data_{count+1:03}_1.png")
    count += 1

In [44]:
cropped = cv2.resize(cropped, (370, 370))

In [82]:
cropped[0]

array([[247, 246, 245, ..., 252, 252, 252],
       [249, 246, 244, ..., 254, 251, 250],
       [248, 244, 244, ..., 252, 251, 247],
       ...,
       [248, 247, 245, ..., 250, 250, 250],
       [249, 248, 247, ..., 250, 250, 250],
       [247, 247, 248, ..., 251, 251, 249]], dtype=uint8)

In [83]:
y = np.bincount(cropped[0])
ii = np.nonzero(y)[0]
zip(ii,y[ii])

ValueError: object too deep for desired array

In [63]:
cv2.imshow('cropped', cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
# generate batch of images
import pathlib
import numpy as np
from pathlib import Path
import cv2

def generate_batch3(path):
    path =Path(path)
    rows = []
    count = 0
    for j in range(2):
        col = []
        for k in range(5):
            white_space = np.ones(shape=(370,370),dtype='uint8')
            white_space.fill(255)
            col.append(white_space)
        rows.append(np.hstack(col))
    for j in range(4):
        col = []
        for k in range(5):
            if k == 0 or k == 4:
                white_space = np.ones(shape=(370,370),dtype='uint8')
                white_space.fill(255)
                col.append(white_space)
            else:
                print(f'data_{count}')                
                col.append(cropped[count])
                count += 1
        rows.append(np.hstack(col))
    for j in range(2):
        col = []
        for k in range(5):
            white_space = np.ones(shape=(370,370),dtype='uint8')
            white_space.fill(255)
            col.append(white_space)
        rows.append(np.hstack(col))
    # rows.append(np.ones((10,rows[0].shape[1]),dtype='uint8'))
    grid = np.vstack(rows)
    Image.fromarray(grid).save(path/f"data_001_012_1.png")
    rows=[]
    return grid
    # Image.fromarray(grid).save("test.png")

In [15]:
grid = generate_batch3('data_x')

data_0
data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8


In [20]:
# Generate 3x3 QR
import pathlib
import numpy as np
from pathlib import Path
import cv2

def generate_batch4(path):
    path =Path(path)
    rows = []
    count = 1    
    col = []
    white_space = np.ones(shape=(540,1550),dtype='uint8')
    white_space.fill(255)
    col.append(white_space)
    rows.append(np.hstack(col))
    for j in range(3):
        col = []
        white_space = np.ones(shape=(370,220),dtype='uint8')
        white_space.fill(255)
        col.append(white_space)
        for k in range(3):            
            print(f'data_{count}')
            img,wtm = make_secureQR(f'data_{count:03}',qr_ver = 3, quant=16)                
            col.append(img)
            count += 1
        white_space = np.ones(shape=(370,220),dtype='uint8')
        white_space.fill(255)
        col.append(white_space)
        rows.append(np.hstack(col))
        
    col = []
    white_space = np.ones(shape=(540,1550),dtype='uint8')
    white_space.fill(255)
    col.append(white_space)
    rows.append(np.hstack(col))
    
    grid = np.vstack(rows)
    Image.fromarray(grid).save(path/f"data_001_009.png")
    rows=[]
    return grid

In [21]:
grid = generate_batch4('data_x')

data_1
data_2
data_3
data_4
data_5
data_6
data_7
data_8
data_9


In [1]:
# Generate 2x2 QR
import pathlib
import numpy as np
from pathlib import Path
import cv2

def generate_batch5(path):
    path =Path(path)
    rows = []
    count = 1    
    col = []
    white_space = np.ones(shape=(470,1180),dtype='uint8')
    white_space.fill(255)
    col.append(white_space)
    rows.append(np.hstack(col))
    for j in range(2):
        col = []
        white_space = np.ones(shape=(370,220),dtype='uint8')
        white_space.fill(255)
        col.append(white_space)
        for k in range(2):            
            print(f'data_{count}')
            img,wtm = make_secureQR(f'data_{count:03}',qr_ver = 3, quant=16)                
            col.append(img)
            count += 1
        white_space = np.ones(shape=(370,220),dtype='uint8')
        white_space.fill(255)
        col.append(white_space)
        rows.append(np.hstack(col))
        
    col = []
    white_space = np.ones(shape=(470,1180),dtype='uint8')
    white_space.fill(255)
    col.append(white_space)
    rows.append(np.hstack(col))
    
    grid = np.vstack(rows)
    Image.fromarray(grid).save(path/f"data_001_004.png")
    rows=[]
    return grid

In [5]:
grid = generate_batch5('2x2QR')

data_1
data_2
data_3
data_4
